![Königsweg Logo](../img/koenigsweg_150.png)

<span style="font-size: small;float: right;">&copy; 2015-2018 Alexander C.S. Hendorf, <a href="http://koenigsweg.com">Königsweg GmbH</a>, Mannheim </span>

---

# Data Wrangling and Visualisation with Juypter and Pandas

---

# Scaling and Optimizing Performance

---

In [1]:
import numpy as np
import pandas as pd
import parquet
import dask
%matplotlib inline

---

### Parquet

---

### Dask